In [1]:
import numpy as np
import math
from gurobi import *
from tabulate import tabulate

import csv
from geopy.geocoders import Nominatim

In [ ]:
# First, we will find the latitudes and longitudes for every game so that we don't have to do this again.
# CAN SKIP FOR FUTURE RUNS
# Reads in the games file
# after running this program once, we can start to use "week1_data_lats_and_longs.csv" as our file name so that we don't
# have to continuously find the latitude and longitude for each game location
game_file_name = "PriorityGames.csv"
game_file = open(game_file_name, "r")
game_csv_reader = csv.reader(game_file)

lists_from_csv = []
for row in game_csv_reader:
    lists_from_csv.append(row)
    
lists_from_csv = lists_from_csv[1:]
    
# CAN SKIP FOR FUTURE RUNS
# Find the latitude and longitude of each game location
for row in lists_from_csv:
    #print(count)
    #count += 1
    geolocator = Nominatim(user_agent="Caitlin")
    location = geolocator.geocode(row[6], timeout=None)
    latitude = location.latitude
    longitude = location.longitude
    row.append(latitude)
    row.append(longitude)
    
    row[2] = int(row[2])
    row[3] = int(row[3])
    
# CAN SKIP FOR FUTURE RUNS
# Writes to a data file (we won't use this)
#the following lines of code write the data to a csv file, which won't be necessary to add it to the model but might
#be good to have for later - we will actually use this so we don't have to continuously find the latitudes and longitudes

header = ['GameDate', 'GameTime', 'WeekNumber', 'GameNumber', 'HomeTeamNumber', 'AwayTeamNumber', 'Location', 'Latitude', 'Longitude']
with open('GameData.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(lists_from_csv)

In [2]:
# Change the priority teams file to have the weights for the team included
team_file_name = "PriorityTeams.csv"
team_file = open(team_file_name, "r")
team_csv_reader = csv.reader(team_file)

team_lists_from_csv = []
for row in team_csv_reader:
    team_lists_from_csv.append(row)
    
# Cut off the label row of the data file so that we don't run into any problems that we don't excpect
team_lists_from_csv = team_lists_from_csv[1:]

In [3]:
# ONLY TO BE RUN IN FUTURE RUNS - need to fix some of this so for now we can just do the longer way
# Reads in the games file
# after running this program once, we can start to use "week1_data_lats_and_longs.csv" as our file name so that we don't
# have to continuously find the latitude and longitude for each game location
game_file_name = "GameData.csv"
game_file = open(game_file_name, "r")
game_csv_reader = csv.reader(game_file)

game_lists_from_csv = []
for row in game_csv_reader:
    game_lists_from_csv.append(row)
    
team_file_name = "TeamWeights.csv"
team_file = open(team_file_name, "r")
team_csv_reader = csv.reader(team_file)

team_lists_from_csv = []
for row in team_csv_reader:
    team_lists_from_csv.append(row)
    
team_lists_from_csv = team_lists_from_csv[1:]

In [245]:
# only look at the first week to start
# week 1 - [1:283]; week 2 - [283:535]; week 3 - [535:742]; week 4 - [742:950]; week 5 - [950:1132]; 
# week 6 - [1132:1293]; week 7 - [1293:1422]; week 8 - [1422:1609]; week 9 - [1609:1815]; week 10 - [1815:2010];
# week 11 - [2010:2212]; week 12 - [2212:2403]; week 13 - [2403:2603]; week 14 - [2603:2800]; week 15 - [2800:2987];
# week 16 - [2987:3182]; week 17 - [3182:3323]

week_game_lists_from_csv = game_lists_from_csv[3182:3323]

In [246]:
# CAN SKIP FOR FUTURE RUNS
# Asssigns each team to a number
teams = []
for row in team_lists_from_csv:
    teams.append(row[0])

team_numbers = {}
number = 0

for team in teams:
    team_numbers[team] = number
    number += 1
    
not_interested_teams = number 

In [247]:
# CAN SKIP FOR FUTURE RUNS
# Using the numbers assigned to the teams from the teams files, assign the same numbers to the teams that appear in
# the games file.
for row in week_game_lists_from_csv:
    home_team = row[4]
    if home_team in team_numbers:
        home_team_number = team_numbers[home_team]
        row[4] = home_team_number
    away_team = row[5]
    if away_team in team_numbers:
        away_team_number = team_numbers[away_team]
        row[5] = away_team_number
        
    row[3] = int(row[3])
        
# If a team in the games file is not a priority team, we will still need to assign it to a number.
for row in week_game_lists_from_csv:
    if type(row[4]) is not int:
        row[4] = number+1
        number += 1
    if type(row[5]) is not int:
        row[5] = number+1
        number += 1
        

In [248]:
# Creates a dictionary for the weight of each team
weights_teams = {}

p1_weight = 5
p2_weight = 3
p3_weight = 1
for i in range(len(team_lists_from_csv)):
    linear_combination = p1_weight*int(team_lists_from_csv[i][1])+p2_weight*int(team_lists_from_csv[i][2])+p3_weight*int(team_lists_from_csv[i][3])
    
    try:
        num_seen = teams_of_interest_seen[team_lists_from_csv[i][0]]
        num_seen = num_seen[0]
        affected_weight = num_seen*30
    except:
        affected_weight = 0
    
    weights_teams[team_lists_from_csv[i][0]] = linear_combination/(affected_weight+1)
    team_lists_from_csv[i].append(weights_teams[team_lists_from_csv[i][0]])
    
    #print(num_seen)
    
print(weights_teams)

{'Alabama': 0.07380073800738007, 'Alabama State': 0.01639344262295082, 'Appalachian State': 0.008264462809917356, 'Arizona': 0.09917355371900827, 'Arizona State': 0.0989010989010989, 'Arkansas': 0.049586776859504134, 'Arkansas State': 0.01639344262295082, 'Auburn': 0.07109004739336493, 'BYU': 0.03278688524590164, 'Baylor': 0.152317880794702, 'Belmont': 0.06593406593406594, 'Binghamton': 0.03225806451612903, 'Boise State': 0.01639344262295082, 'Brown': 0.01098901098901099, 'Buffalo': 0.04918032786885246, 'Butler': 0.03278688524590164, 'CS Fullerton': 0.01098901098901099, 'California': 0.01639344262295082, 'California Baptist': 0.03225806451612903, 'Charlotte': 0.004739336492890996, 'Chattanooga': 0.03225806451612903, 'Cincinnati': 0.026490066225165563, 'Clemson': 0.01639344262295082, 'Coastal Carolina': 0.01639344262295082, 'College of Charleston': 0.03225806451612903, 'Colorado': 0.06557377049180328, 'Colorado State': 0.09836065573770492, 'Connecticut': 0.052132701421800945, 'Creighton

In [249]:
# Initialize some variables
games_for_teams = {}
teams_in_games = {}
L = {}

In [250]:
# create variables for scout locations and maximum distances allowed
# (there are 5 scouts)
num_scouts = 5
scout_locations = np.array([[35.2271,-80.8431],[35.2271,-80.8431],
                            [38.9072,-77.0369],[40.7128,-74.0060],
                            [34.0522,-118.2437]])

scout_distances = np.array([2000,2000,2000,2000,2000])

num_games = len(week_game_lists_from_csv)
num_teams_of_interest = len(team_lists_from_csv)
num_teams = number

In [251]:
# Creates the games_for_teams dictionary
for j in range(len(week_game_lists_from_csv)):
    team_1 = week_game_lists_from_csv[j][4]
    team_2 = week_game_lists_from_csv[j][5]
    if team_1 not in games_for_teams:
        games_for_teams[team_1] = [int(week_game_lists_from_csv[j][3])]
    else:
        games_for_teams[team_1].append(int(week_game_lists_from_csv[j][3]))
    if team_2 not in games_for_teams:
        games_for_teams[team_2] = [int(week_game_lists_from_csv[j][3])]
    else:
        games_for_teams[team_2].append(int(week_game_lists_from_csv[j][3]))
        
    #the following line of code creates the teams in games dictionary
    teams_in_games[int(week_game_lists_from_csv[j][3])]=[team_1,team_2]
        
    #the following lines of code create the set L where each subset has overlapping games
    date = week_game_lists_from_csv[j][0]
    # If we get a problem where the program thinks L is a list, we have to restart the kernel because later in the
    # code we turn L into a list, which prohibits the following lines of code from working.
    if date not in L:
        L[date] = [int(week_game_lists_from_csv[j][3])]
    else:
        L[date].append(int(week_game_lists_from_csv[j][3]))

In [252]:
# Creates a list of the latitudes and longitudes for each game
game_locations = []

for game in week_game_lists_from_csv:
    game_locations.append([float(game[7]),float(game[8])])

#print(week_game_lists_from_csv[0])

In [253]:
#the following lines of code continue creating the set L where each subset has overlapping games
values = L.values()
L = list(values)

In [254]:
# Creates the weights for each game using the team weights    
weights_games = []
# Creates a list of game-player combinations that are valid
lst = []

weights_team_numbers = {}
for team in team_numbers.keys():
    numb = team_numbers.get(team)
    weights_team_numbers[numb] = weights_teams[team]
for i in range(num_teams_of_interest, num_teams):
    weights_team_numbers[i] = 0

for game in teams_in_games.keys():
    teams = teams_in_games.get(game)
    # create a list of tuples with valid game-player combinations
    for t in teams:
        gt_tuple = (int(game),t)
        lst.append(gt_tuple)
    team_1 = teams[0]
    team_2 = teams[1]
    
    #print(weights_team_numbers)
    if team_1 in weights_team_numbers and team_2 in weights_team_numbers:
        weights_games.append(weights_team_numbers[team_1]+weights_team_numbers[team_2])
    elif team_1 in weights_team_numbers and team_2 not in weights_team_numbers:
        weights_games.append(weights_team_numbers[team_1])
    elif team_2 in weights_team_numbers and team_1 not in weights_team_numbers:
        weights_games.append(weights_team_numbers[team_2])
    #else:
     #   weights_games.append(0) 

## Calculate distance between two locations

In [255]:
def degrees2radians(lat, long):
    lat = lat*np.pi/180
    long = long*np.pi/180
    return lat,long

#function returns distance between two latitudes and longitudes, in km
def distance(lat_scout, long_scout, lat_game, long_game):
    r = 6371 # mean radius of the Earth in km
    lat_scout, long_scout = degrees2radians(lat_scout,long_scout)
    lat_game, long_game = degrees2radians(lat_game,long_game)
    del_lambda = long_scout-long_game
    del_sigma = lat_scout-lat_game
    sigma_m = (lat_scout+lat_game)/2
    x = del_lambda*np.cos(sigma_m)
    y = del_sigma
    d = r*math.sqrt(x**2+y**2)
    return d

## Create our Gurobi BILP

In [256]:
# create new model
model = Model('Simple')

# create variables
x = model.addVars(num_scouts, num_games, vtype=GRB.BINARY, name='x')

# We only need to make y_gp variables for the corresponding game/player combinations
y_gt = model.addVars(lst, vtype=GRB.BINARY, name='y_gt')

y_t = model.addVars(num_teams+1, vtype=GRB.BINARY, name='y_t')


## Create the constraints

In [257]:
# constraint (1)
# for every game, sum over scouts
for g in range(num_games):
    #print(g)
    model.addConstr(quicksum(x[s,g] for s in range(num_scouts)) <= 1)
    
# constraint (2)     
# for every scout, sum over games
for s in range(num_scouts):
    #print(s)
    model.addConstr(quicksum(distance(scout_locations[s][0],scout_locations[s][1],game_locations[g][0],game_locations[g][1])*x[s,g] for g in range(num_games))
                    <= scout_distances[s])
    
# constraint (3)
# need to make sure to start at game 0 to align with indices
for s in range(num_scouts):
    for subset in L:
        model.addConstr(quicksum(x[s,g] for g in subset) <= 1)

# constraint (4)
# let w=weight of the game, set up an if statement to solve for this for each game based on the number of players of interest playing
for s in range(num_scouts):
    for g in teams_in_games.keys():
        model.addConstr(x[s,g] <= quicksum(y_gt[g,t] for t in teams_in_games.get(g)))

# constraint (5)
for g in teams_in_games.keys():
    for t in teams_in_games.get(g):
        model.addConstr(y_gt[g,t] <= quicksum(x[s,g] for s in range(num_scouts)))

# constraint (6)
for t in games_for_teams.keys():
    for g in games_for_teams.get(t):
        model.addConstr(y_gt[g,t] <= y_t[t])

# constraint (7)
for t in games_for_teams.keys():
    model.addConstr(y_t[t] <= quicksum(y_gt[g,t] for g in games_for_teams.get(t)))

# objective function
total_distance=sum(scout_distances)

m = quicksum(distance(scout_locations[s][0],scout_locations[s][1],game_locations[g][0],game_locations[g][1])*x[s,g]
                             for s in range(num_scouts) for g in range(num_games))
model.setObjective(m-total_distance*(quicksum(weights_team_numbers[t]*y_t[t] for t in range(num_teams))), GRB.MINIMIZE)



In [258]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1620 rows, 1190 columns and 6938 nonzeros
Model fingerprint: 0x9b3a4cd4
Variable types: 0 continuous, 1190 integer (1190 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+03]
  Objective range  [1e-02, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -11607.66913
Presolve removed 422 rows and 352 columns
Presolve time: 0.02s
Presolved: 1198 rows, 838 columns, 4901 nonzeros
Variable types: 0 continuous, 838 integer (838 binary)

Root relaxation: objective -2.935195e+04, 429 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -29351.948    0   16 -11607.669 -29351.948   153%     -    0s
H    0     0                    -28412.2995

In [ ]:
model.write('simple_model_week17.lp')

In [243]:
scouts_games_watched = {}

for v in model.getVars():
    if 'x' in v.varName and v.x != 0:
        scout = v.varName[2]
        game = v.varName[4:-1]
        if scout not in scouts_games_watched:
            scouts_games_watched[scout] = [game]
        else:
            scouts_games_watched[scout].append(game)
            
final_list = []
for scout_key in scouts_games_watched.keys():
    scout_key_games = scouts_games_watched.get(scout_key)
    if scout_key == '0' or scout_key == '1':
        final_scout_location = 'Charlotte, NC'
    elif scout_key == '2':
        final_scout_location = 'Washington, DC'
    elif scout_key == '3':
        final_scout_location = 'New York, NY'
    elif scout_key == '4':
        final_scout_location = 'Los Angeles, CA'
        
    for each_game in scout_key_games:
        final_list.append([scout_key, final_scout_location, each_game])
  #  internal_list[0] = scout_key
   
team_names = list(team_numbers.keys())
team_numbers_ints = list(team_numbers.values())
    
for i in range(len(final_list)):
    game_number = int(final_list[i][2])
    game_row = week_game_lists_from_csv[game_number]
    
    #print(game_row)
    
    final_list[i].append(week_game_lists_from_csv[game_number][6])
    final_list[i].append(week_game_lists_from_csv[game_number][0])
    
    team_1_number = week_game_lists_from_csv[game_number][4]
    team_2_number = week_game_lists_from_csv[game_number][5]
    
    #print(team_1_number, team_2_number)
    
    if team_1_number in team_numbers_ints and team_2_number in team_numbers_ints:
        final_list[i].append(team_names[team_1_number])
        final_list[i].append(team_names[team_2_number])
    elif team_1_number in team_numbers_ints and team_2_number not in team_numbers_ints:
        final_list[i].append(team_names[team_1_number])
        final_list[i].append('Not a priority team')
    elif team_1_number not in team_numbers_ints and team_2_number in team_numbers_ints:
        final_list[i].append('Not a priority team')
        final_list[i].append(team_names[team_2_number])
        
    final_list[i].append(weights_games[game_number])
    
    
#print(final_list)

In [ ]:
table = final_list
headers = ['Scout Number', 'Scout Location', 'Game Number', 'Game Location', 'Date', 'Team 1', 'Team 2', 'Game Weight']

print(tabulate(table, headers, tablefmt="fancy_grid"))

with open('week17.txt', 'w') as f:
    f.write(tabulate(table, headers))



In [19]:
# only run for the first week
teams_of_interest_seen = {}
for team in team_numbers.keys():
    teams_of_interest_seen[team] = [0, weights_teams[team]]


In [244]:
for game_seen in final_list:
    team_1_seen = game_seen[5]
    team_2_seen = game_seen[6]
    
    if team_1_seen != 'Not a priority team':
        teams_of_interest_seen[team_1_seen][0] += 1
    if team_2_seen != 'Not a priority team':
        teams_of_interest_seen[team_2_seen][0] += 1
        
print(teams_of_interest_seen)

{'Alabama': [9, 20.0], 'Alabama State': [2, 1.0], 'Appalachian State': [4, 1.0], 'Arizona': [4, 12.0], 'Arizona State': [3, 9.0], 'Arkansas': [4, 6.0], 'Arkansas State': [2, 1.0], 'Auburn': [7, 15.0], 'BYU': [2, 2.0], 'Baylor': [5, 23.0], 'Belmont': [3, 6.0], 'Binghamton': [1, 1.0], 'Boise State': [2, 1.0], 'Brown': [3, 1.0], 'Buffalo': [2, 3.0], 'Butler': [2, 2.0], 'CS Fullerton': [3, 1.0], 'California': [2, 1.0], 'California Baptist': [1, 1.0], 'Charlotte': [7, 1.0], 'Chattanooga': [1, 1.0], 'Cincinnati': [5, 4.0], 'Clemson': [2, 1.0], 'Coastal Carolina': [2, 1.0], 'College of Charleston': [1, 1.0], 'Colorado': [2, 4.0], 'Colorado State': [2, 6.0], 'Connecticut': [7, 11.0], 'Creighton': [3, 3.0], 'Davidson': [6, 5.0], 'Dayton': [4, 6.0], 'Depaul': [6, 11.0], 'Detroit Mercy': [3, 4.0], 'Duke': [11, 25.0], 'Duquesne': [1, 1.0], 'East Carolina': [3, 1.0], 'Eastern Kentucky': [1, 1.0], 'Evansville': [1, 1.0], 'Florida': [5, 8.0], 'Florida A&M': [1, 1.0], 'Florida Atlantic': [1, 1.0], 'Fl

In [ ]:
# Print the names of the columns.
print ("{:<25} {:<25} {:<25}".format('TEAM NAME', 'NUMBER OF TIMES SEEN', 'ORIGINAL WEIGHT'))    

for key, value in teams_of_interest_seen.items():
    team_interest = key
    num_times_seen, original_weight = value
    print ("{:<25} {:<25} {:<25}".format(team_interest, num_times_seen, original_weight))


In [ ]:
with open('week9_dict.txt', 'w') as f:
    f.write("{:<25} {:<25} {:<25}".format('TEAM NAME', 'NUMBER OF TIMES SEEN', 'ORIGINAL WEIGHT'))  
    f.write('\n')
    for key, value in teams_of_interest_seen.items():
        team_interest = key
        num_times_seen, original_weight = value
        f.write("{:<25} {:<25} {:<25}".format(team_interest, num_times_seen, original_weight))
        f.write('\n')